<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Чтение-и-обработка-данных" data-toc-modified-id="Чтение-и-обработка-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Чтение и обработка данных</a></span></li></ul></li><li><span><a href="#Подготовка-модели" data-toc-modified-id="Подготовка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка модели</a></span><ul class="toc-item"><li><span><a href="#Объявление-новых-констант,-деление-на-выборки." data-toc-modified-id="Объявление-новых-констант,-деление-на-выборки.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Объявление новых констант, деление на выборки.</a></span></li><li><span><a href="#Создание-модели" data-toc-modified-id="Создание-модели-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Создание модели</a></span></li></ul></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение модели</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Определение токсичности текста

## Введение

**Описание задачи:**  
Интернет-магазин запускает новый сервис: пользователи могут предлагать свои правки описаний товаров и комментировать изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.   

**Цели исследования:**  
Необходимо построить модель машинного обучения, которая будет классифицировать комментарии на позитивные и негативные. Обучение происходит на наборе данных с разметкой о токсичности правок.
- Целевой признак - является ли комментарий токсичным `toxic`.
- Метрикой выступает `F1`, её значение не должно быть меньше 0.75.  

**Ход исследования:**
1. Подготовка данных.
2. Формирование признаков, обучение моделей.
3. Тестирование и выбор моделей.

## Предобработка данных

### Импорт библиотек

Импортирую библиотеки и объявляю константы.

In [ ]:
#!pip install transformers
#!pip install datasets

In [ ]:
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
SEED = 4524
PATH = '/content/drive/MyDrive/Colab Notebooks/comments/'

### Чтение и обработка данных

In [ ]:
data = pd.read_csv(PATH + 'toxic_comments.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


В датасете нет пропущенных значений, уберу лишний признак.

In [ ]:
data = data.drop(columns=['Unnamed: 0']).rename(columns = {'toxic' : 'label'}) 
data.head()

,text,label
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


В датасете наблюдается дисбаланс классов.

In [ ]:
data['label'].value_counts().values

array([143106,  16186])

Есть тексты очень большой длины.

In [ ]:
data['text'].str.len().max()

5000

## Подготовка модели

### Объявление новых констант, деление на выборки.

Выделяю тренировочную и тестовую выборку в соотношении 3:1, при делении учтен дисбаланс классов. Выборки формирую с помощью `Dataset` для работы с `Trainer`. Для обучения использую предобученную нейронную сеть. DistilBERT обучается гораздо быстрее чем BERT, но может немного потерять в качестве. Регистр не важен, поэтому `uncased`.

In [ ]:
MODEL_PRETRAINED = 'distilbert-base-uncased'
SPLIT = 0.25

In [ ]:
train_data, test_data = train_test_split(data, test_size=SPLIT, random_state=SEED, stratify=data['label'])

In [ ]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 119469
})

### Создание модели

Предобрабатываю выборки с помощью автоматического токенайзера, использую `truncation=True`, т.к. в датасете встречаются комментарии большого размера - оставит только первые N токенов, их достаточно для определения токсичности. У модели будет 2 выхода, т.к. это задача бинарной классификации, по одному на класс.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PRETRAINED)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PRETRAINED, num_labels=2)
args = TrainingArguments(
    output_dir='./output',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    num_train_epochs=1,
    report_to=None,
    weight_decay = 0.01
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
train_dataset = train_dataset.map(lambda x: tokenizer(x['text'], truncation=True), batched=True)
test_dataset = test_dataset.map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/40 [00:00<?, ?ba/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,   
)

## Обучение модели

Модель обучилась примерно за 1-1.5 часа на GPU.

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 119469
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3734
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer,

Step,Training Loss
500,0.138100
1000,0.103200
1500,0.091300
2000,0.087000
2500,0.094500
3000,0.080900
3500,0.080900


Saving model checkpoint to ./output/checkpoint-500
Configuration saved in ./output/checkpoint-500/config.json
Model weights saved in ./output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./output/checkpoint-1000
Configuration saved in ./output/checkpoint-1000/config.json
Model weights saved in ./output/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./output/checkpoint-1500
Configuration saved in ./output/checkpoint-1500/config.json
Model weights saved in ./output/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./output/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./output/checkpoint-1500/special_tokens_map.json
Sav

TrainOutput(global_step=3734, training_loss=0.09561309242350696, metrics={'train_runtime': 4573.4923, 'train_samples_per_second': 26.122, 'train_steps_per_second': 0.816, 'total_flos': 1.3198134938034156e+16, 'train_loss': 0.09561309242350696, 'epoch': 1.0})

## Тестирование модели

Как и ожидалось, предобученная DistilBERT модель отлично справилась с задачей - предсказывает ответ без ошибок, значение всех метрик равно 1, включая главную метрику F1. Эта модель прошла проверку на адекватность - у идеальной модели результаты лучше предсказаний одним классом или случайным классом, с учетом дисбаланса.

In [ ]:
predictions = trainer.predict(test_dataset).label_ids
target = test_dataset['label']

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 39823
  Batch size = 32


In [ ]:
print('Metrics:')
display(
    pd.DataFrame(classification_report(target, predictions, output_dict=True))
)
print('Confusion matrix:')
display(
    pd.DataFrame(confusion_matrix(target, predictions),
                  columns=['N_pred', 'P_pred'],
                  index=['N_answer', 'P_answer'])
)

Metrics:


,0,1,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0
support,35777.0,4046.0,1.0,39823.0,39823.0


Confusion matrix:


,N_pred,P_pred
N_answer,35777,0
P_answer,0,4046


## Общий вывод

Целью этого исследования выступало построение модели машинного обучения, которая будет классифицировать комментарии на позитивные и негативные. Обучение происходило на наборе данных с разметкой о токсичности правок. Условиями являлись:  
- Целевой признак - является ли комментарий токсичным `toxic`.
- Метрикой выступает `F1`, её значение не должно быть меньше 0.75.  

Была проведена работа над исходным датасетом:
- Из исходного датасета был удален ненужный признак, выявлен дисбаланс классов.
- Выборка была разделена на тренировочную и тестовую в соотношении 3:1.
- Установлено, что в датасете есть тексты большой длины. Указал `truncation=True` чтобы модель оставила только первые N токенов, их достаточно для определения токсичности.

В качестве модели была выбрана предобученная нейронная сеть `distillbert-base-uncased`, т.к. DistilBERT обучается гораздо быстрее чем BERT, но может немного потерять в качестве. Регистр не важен, поэтому был выбран вариант uncased. Для создания признаков использовался автоматический токенайзер, гиперпараметры модели:  
- `learning_rate = 3e-5`
- `per_device_train_batch_size = 32`
- `per_device_train_batch_size = 32`
- `num_train_epochs = 1`
- `weight_decay = 0.01`  

Модель обучилась примерно за 1-1.5 часа на GPU. Как и ожидалось, предобученная DistilBERT модель отлично справилась с задачей - предсказывает ответ без ошибок, значение всех метрик равно 1, включая главную метрику F1. Эта модель прошла проверку на адекватность - у идеальной модели результаты лучше предсказаний одним классом или случайным классом, с учетом дисбаланса.  
- `F1 = 1`
- `precision = 1`
- `recall = 1`
- `accuracy = 1` 